In [11]:
# Cell 1: Imports and Data Loading
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

def load_data():
    # Load all datasets
    k_percentage_df = pd.read_csv('team_strikeout_percentage.csv')
    pitcher_data = pd.read_csv('pitchers_data.csv')
    betting_data = pd.read_csv('betting_data.csv')
    
    # Merge and preprocess data
    pitcher_data = pitcher_data.merge(
        betting_data[['Name_abbreviation', 'Team']], 
        left_on='Pitcher', 
        right_on='Name_abbreviation', 
        how='left'
    )
    
    # Feature engineering
    pitcher_data['SO_per_IP'] = pitcher_data['SO'] / pitcher_data['IP']
    pitcher_data['BB_per_IP'] = pitcher_data['BB'] / pitcher_data['IP']
    pitcher_data['K-BB%'] = pitcher_data['SO_per_IP'] - pitcher_data['BB_per_IP']
    
    # Merge with team stats
    pitcher_data = pitcher_data.merge(k_percentage_df, on='Team', how='left')
    pitcher_data.rename(columns={'%K': 'Team_K%'}, inplace=True)
    pitcher_data = pitcher_data.merge(
        k_percentage_df.rename(columns={'%K': 'Opp_K%'}), 
        left_on='Opp', right_on='Team', how='left'
    )
    
    # Handle missing values
    pitcher_data.fillna({
        'SO_per_IP': pitcher_data['SO_per_IP'].mean(),
        'BB_per_IP': pitcher_data['BB_per_IP'].mean(),
        'Team_K%': pitcher_data['Team_K%'].mean(),
        'Opp_K%': pitcher_data['Opp_K%'].mean()
    }, inplace=True)
    
    return pitcher_data, k_percentage_df

pitchers_df, k_percentage_df = load_data()

In [15]:
# Cell 2: Performance Calculation Function
def calculate_weighted_performance(pitcher_data, current_season, last_season=None):
    current_season_data = pitcher_data[pitcher_data['Season'] == current_season]
    last_5_games = current_season_data.tail(5)
    last_10_games = current_season_data.tail(10)
    
    # Calculate rolling averages
    current_season_data['SO_rolling_5'] = current_season_data['SO'].rolling(5).mean()
    current_season_data['SO_rolling_10'] = current_season_data['SO'].rolling(10).mean()
    
    # Calculate home/away splits
    home_stats = current_season_data[current_season_data['Home'] == 1.0].mean(numeric_only=True)
    away_stats = current_season_data[current_season_data['Home'] == 0.0].mean(numeric_only=True)

    if last_season is not None:
        last_season_data = pitcher_data[pitcher_data['Season'] == last_season]
        weight_current_season = 0.40
        weight_last_5_games = 0.25
        weight_last_10_games = 0.15
        weight_last_season = 0.20
    else:
        last_season_data = pd.DataFrame()
        weight_current_season = 0.50
        weight_last_5_games = 0.30
        weight_last_10_games = 0.20
        weight_last_season = 0.0

    metrics = ['IP', 'H', 'BB', 'ERA', 'FIP', 'SO', 'SO_rolling_5', 'SO_rolling_10']
    weighted_values = {}

    for metric in metrics:
        current_mean = current_season_data[metric].mean() if not current_season_data.empty else 0
        last_5_mean = last_5_games[metric].mean() if not last_5_games.empty else 0
        last_10_mean = last_10_games[metric].mean() if not last_10_games.empty else 0
        last_season_mean = last_season_data[metric].mean() if not last_season_data.empty else 0

        weighted_values[metric] = (
            weight_current_season * current_mean +
            weight_last_5_games * last_5_mean +
            weight_last_10_games * last_10_mean +
            weight_last_season * last_season_mean
        )
    
    # Add home/away splits
    weighted_values['Home_IP'] = home_stats.get('IP', 0)
    weighted_values['Away_IP'] = away_stats.get('IP', 0)
    weighted_values['Home_SO'] = home_stats.get('SO', 0)
    weighted_values['Away_SO'] = away_stats.get('SO', 0)
    weighted_values['Home'] = current_season_data['Home'].mean()
    
    return weighted_values

In [33]:
def train_model(pitchers_df, k_percentage_df):
    # Calculate weighted performance for each pitcher
    weighted_pitcher_data = []
    for pitcher in pitchers_df['Pitcher'].unique():
        pitcher_data = pitchers_df[pitchers_df['Pitcher'] == pitcher].copy()
        
        # Ensure we have the required columns for rolling calculations
        if 'SO' not in pitcher_data.columns:
            pitcher_data['SO'] = 0  # Default value if strikeouts data is missing
            
        # Calculate rolling averages
        pitcher_data = pitcher_data.sort_values('Season')
        pitcher_data['SO_rolling_5'] = pitcher_data['SO'].rolling(5, min_periods=1).mean()
        pitcher_data['SO_rolling_10'] = pitcher_data['SO'].rolling(10, min_periods=1).mean()
        
        # Calculate home/away stats
        pitcher_data['Home_IP'] = pitcher_data[pitcher_data['Home'] == 1.0]['IP'].mean()
        pitcher_data['Away_IP'] = pitcher_data[pitcher_data['Home'] == 0.0]['IP'].mean()
        pitcher_data['Home_SO'] = pitcher_data[pitcher_data['Home'] == 1.0]['SO'].mean()
        pitcher_data['Away_SO'] = pitcher_data[pitcher_data['Home'] == 0.0]['SO'].mean()
        
        performance = calculate_weighted_performance(pitcher_data, current_season=2024, last_season=2023)
        performance['Pitcher'] = pitcher
        performance['Opp_K%'] = pitcher_data['Opp_K%'].iloc[0] if not pitcher_data.empty else k_percentage_df['%K'].mean()
        performance['Team_K%'] = pitcher_data['Team_K%'].iloc[0] if not pitcher_data.empty else k_percentage_df['%K'].mean()
        weighted_pitcher_data.append(performance)

    weighted_df = pd.DataFrame(weighted_pitcher_data)
    
    # Add additional features with safety checks
    weighted_df['IP'] = weighted_df['IP'].replace(0, 1)  # Avoid division by zero
    weighted_df['SO_per_IP'] = weighted_df['SO'] / weighted_df['IP']
    weighted_df['BB_per_IP'] = weighted_df['BB'] / weighted_df['IP']
    weighted_df['K-BB%'] = weighted_df['SO_per_IP'] - weighted_df['BB_per_IP']
    
    # Define required features
    required_features = [
        'IP', 'H', 'BB', 'ERA', 'FIP', 'SO_per_IP', 'BB_per_IP', 'K-BB%', 
        'Opp_K%', 'Team_K%', 'Home', 'SO_rolling_5', 'SO_rolling_10',
        'Home_IP', 'Away_IP', 'Home_SO', 'Away_SO'
    ]
    
    # Ensure all features exist
    for feature in required_features:
        if feature not in weighted_df.columns:
            weighted_df[feature] = 0  # Initialize missing features with 0
            print(f"Warning: Initialized missing feature {feature} with zeros")
    
    X = weighted_df[required_features].fillna(0)
    y = weighted_df['SO']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define models to test
    models = {
        'RandomForest': RandomForestRegressor(random_state=42),
        'XGBoost': XGBRegressor(random_state=42),
        'LightGBM': LGBMRegressor(random_state=42),
        'GradientBoosting': GradientBoostingRegressor(random_state=42)
    }

    best_model = None
    best_score = -np.inf
    best_model_name = ""
    results = {}

    # Evaluate each model
    for name, model in models.items():
        pipeline = make_pipeline(StandardScaler(), model)
        
        # Cross-validation
        scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
        avg_score = np.mean(scores)
        
        # Train and evaluate
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        test_r2 = r2_score(y_test, y_pred)
        test_mae = mean_absolute_error(y_test, y_pred)
        
        results[name] = {
            'CV_R2': avg_score,
            'Test_R2': test_r2,
            'Test_MAE': test_mae
        }
        
        print(f"{name:15} | CV R2: {avg_score:.4f} | Test R2: {test_r2:.4f} | Test MAE: {test_mae:.4f}")
        
        if avg_score > best_score:
            best_score = avg_score
            best_model = pipeline
            best_model_name = name

    print(f"\nBest model: {best_model_name} with R2 score: {best_score:.4f}")
    return best_model, results

# Train the model
model, model_results = train_model(pitchers_df, k_percentage_df)

RandomForest    | CV R2: 0.5167 | Test R2: 0.8060 | Test MAE: 0.2382
XGBoost         | CV R2: -3.2177 | Test R2: 0.7716 | Test MAE: 0.3181
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 16, number of used features: 0
[LightGBM] [Info] Start training from score 5.752646
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there ar

In [36]:
def predict_strikeouts_with_confidence(model, pitchers_df, k_percentage_df, pitcher_name, opponent_team, strikeout_line):
    """
    Make prediction with confidence metrics for a pitcher vs opponent.
    
    Args:
        model: Trained ML pipeline
        pitchers_df: Pitchers historical data
        k_percentage_df: Team strikeout percentages
        pitcher_name: Pitcher abbreviation
        opponent_team: Opponent team abbreviation
        strikeout_line: Betting line for strikeouts
        
    Returns:
        Dictionary with prediction, recommendation and confidence metrics
        or None if prediction couldn't be made
    """
    # Create a copy to avoid SettingWithCopyWarning
    pitcher_data = pitchers_df[pitchers_df['Pitcher'] == pitcher_name].copy()
    
    if pitcher_data.empty:
        print(f"No data found for pitcher: {pitcher_name}")
        return None

    # Ensure we have required columns for rolling calculations
    required_columns = ['Season', 'SO', 'IP', 'Home']
    for col in required_columns:
        if col not in pitcher_data.columns:
            pitcher_data[col] = 0  # Initialize with default value
            
    # Calculate rolling features with proper sorting
    pitcher_data = pitcher_data.sort_values('Season')
    pitcher_data['SO_rolling_5'] = pitcher_data['SO'].rolling(5, min_periods=1).mean()
    pitcher_data['SO_rolling_10'] = pitcher_data['SO'].rolling(10, min_periods=1).mean()
    
    # Calculate home/away splits
    pitcher_data['Home_IP'] = pitcher_data[pitcher_data['Home'] == 1.0]['IP'].mean()
    pitcher_data['Away_IP'] = pitcher_data[pitcher_data['Home'] == 0.0]['IP'].mean()
    pitcher_data['Home_SO'] = pitcher_data[pitcher_data['Home'] == 1.0]['SO'].mean()
    pitcher_data['Away_SO'] = pitcher_data[pitcher_data['Home'] == 0.0]['SO'].mean()

    # Get opponent strikeout rate
    opponent_k = k_percentage_df.loc[k_percentage_df['Team'] == opponent_team, '%K'].mean()
    if np.isnan(opponent_k):
        opponent_k = k_percentage_df['%K'].mean()  # Fallback to league average
    
    # Calculate weighted performance metrics
    performance = calculate_weighted_performance(
        pitcher_data=pitcher_data,
        current_season=2024,
        last_season=2023 if not pitcher_data[pitcher_data['Season'] == 2023].empty else None
    )
    
    # Prepare features DataFrame
    features = pd.DataFrame([performance])
    
    # Add derived features with safety checks
    features['IP'] = features['IP'].replace(0, 1)  # Avoid division by zero
    features['SO_per_IP'] = features['SO'] / features['IP']
    features['BB_per_IP'] = features['BB'] / features['IP']
    features['K-BB%'] = features['SO_per_IP'] - features['BB_per_IP']
    features['Opp_K%'] = opponent_k
    features['Team_K%'] = pitcher_data['Team_K%'].iloc[0] if not pitcher_data.empty else k_percentage_df['%K'].mean()
    
    # Define and validate all required features
    model_features = [
        'IP', 'H', 'BB', 'ERA', 'FIP', 'SO_per_IP', 'BB_per_IP', 'K-BB%', 
        'Opp_K%', 'Team_K%', 'Home', 'SO_rolling_5', 'SO_rolling_10',
        'Home_IP', 'Away_IP', 'Home_SO', 'Away_SO'
    ]
    
    # Ensure all features exist in the DataFrame
    for feature in model_features:
        if feature not in features.columns:
            features[feature] = 0  # Initialize missing features with 0
            print(f"Warning: Initialized missing feature {feature} with zeros")
    
    input_features = features[model_features].fillna(0)
    
    try:
        # Make prediction
        predicted_strikeouts = model.predict(input_features)[0]
        
        # Calculate confidence metrics
        if hasattr(model, 'named_steps') and 'randomforestregressor' in model.named_steps:
            # For RandomForest - use tree variance
            predictions = [tree.predict(input_features) for tree in 
                         model.named_steps.randomforestregressor.estimators_]
            std_dev = np.std(predictions)
            confidence = max(0, min(1 - (std_dev / 3), 1))
        else:
            # For other models - use simple confidence based on line proximity
            std_dev = 0
            confidence = 0.8 - (abs(predicted_strikeouts - strikeout_line) / 10)
            confidence = max(0, min(confidence, 1))
        
        # Determine recommendation
        recommended_side = "Over" if predicted_strikeouts > strikeout_line else "Under"
        
        return {
            'predicted_value': float(predicted_strikeouts),
            'recommended_side': recommended_side,
            'confidence_percentage': float(confidence * 100),
            'std_dev': float(std_dev)
        }
        
    except Exception as e:
        print(f"Prediction failed for {pitcher_name}: {str(e)}")
        return None
    

In [37]:
def process_betting_data(model, pitchers_df, k_percentage_df, betting_data_path):
    """
    Process betting data and add predictions using the trained model.
    
    Args:
        model: Trained ML pipeline
        pitchers_df: Pitchers historical data
        k_percentage_df: Team strikeout percentages
        betting_data_path: Path to betting data CSV
        
    Returns:
        Updated betting DataFrame with predictions
    """
    # Load betting data
    betting_data = pd.read_csv(betting_data_path)
    
    # Initialize prediction columns
    betting_data['ML Strikeout Line'] = (betting_data['Over Line'] + betting_data['Under Line']) / 2
    betting_data['ML Predict Value'] = np.nan
    betting_data['ML Recommend Side'] = np.nan
    betting_data['ML Confidence Percentage'] = np.nan
    betting_data['Pitcher 2023'] = False
    
    def has_2023_data(pitcher_name):
        """Check if pitcher has 2023 data available."""
        return not pitchers_df[(pitchers_df['Pitcher'] == pitcher_name) & 
                             (pitchers_df['Season'] == 2023)].empty
    
    for index, row in betting_data.iterrows():
        pitcher_name = row['Name_abbreviation']
        opponent_team = row['Opponent']
        strikeout_line = row['ML Strikeout Line']
        
        pitcher_2023 = has_2023_data(pitcher_name)
        betting_data.at[index, 'Pitcher 2023'] = pitcher_2023
        
        # Make prediction
        result = predict_strikeouts_with_confidence(
            model=model,
            pitchers_df=pitchers_df,
            k_percentage_df=k_percentage_df,
            pitcher_name=pitcher_name,
            opponent_team=opponent_team,
            strikeout_line=strikeout_line
        )
        
        if result:
            betting_data.at[index, 'ML Predict Value'] = result['predicted_value']
            betting_data.at[index, 'ML Recommend Side'] = result['recommended_side']
            betting_data.at[index, 'ML Confidence Percentage'] = result['confidence_percentage']
            
            # Print progress
            print(f"{pitcher_name} vs {opponent_team}: "
                  f"Line {strikeout_line:.1f} → Pred {result['predicted_value']:.1f} "
                  f"({result['recommended_side']}, {result['confidence_percentage']:.0f}%)")
        else:
            print(f"⚠️ No prediction for {pitcher_name} vs {opponent_team} - missing data")
    
    return betting_data

In [39]:
# Execute the full pipeline
if __name__ == "__main__":
    # Load data and train model
    pitchers_df, k_percentage_df = load_data()
    model, model_results = train_model(pitchers_df, k_percentage_df)
    
    # Process betting data
    updated_betting_data = process_betting_data(
        model=model,
        pitchers_df=pitchers_df,
        k_percentage_df=k_percentage_df,
        betting_data_path='betting_data.csv'
    )
    
    # Save results
    updated_betting_data.to_csv('betting_data_with_predictions.csv', index=False)
    print("\n✅ Predictions saved to betting_data_with_predictions.csv")
    
    # Show top 10 most confident predictions
    print("\nTop 10 Most Confident Predictions:")
    display(updated_betting_data.sort_values('ML Confidence Percentage', ascending=False).head(10))
    
    # Show model comparison results
    print("\nModel Comparison Results:")
    model_comparison = pd.DataFrame(model_results).T
    display(model_comparison.sort_values('CV_R2', ascending=False))

RandomForest    | CV R2: 0.5167 | Test R2: 0.8060 | Test MAE: 0.2382
XGBoost         | CV R2: -3.2177 | Test R2: 0.7716 | Test MAE: 0.3181
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 16, number of used features: 0
[LightGBM] [Info] Start training from score 5.752646
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there ar

,Player,Name_abbreviation,Team,Opponent,Home Team,Away Team,Over Line,Over Odds,Under Line,Under Odds,...,Streak,Streak Type,Probability,Bet Rating,Diff,ML Strikeout Line,ML Predict Value,ML Recommend Side,ML Confidence Percentage,Pitcher 2023
14,Paul Skenes,skenepa,PIT,NaN,NaN,NaN,6.5,1.62,7.5,1.62,...,2,under,NaN,NaN,0.56,7.00,7.029933,Over,79.700669,False
18,Yusei Kikuchi,kikucyu,LAA,NaN,NaN,NaN,6.5,2.10,6.5,1.87,...,1,under,NaN,NaN,-0.41,6.50,6.445246,Under,79.452462,True
22,Framber Valdez,valdefr,HOU,NaN,NaN,NaN,5.5,1.71,6.0,1.73,...,3,under,NaN,NaN,0.25,5.75,5.806969,Over,79.430307,True
23,Hunter Greene,greenhu,CIN,NaN,NaN,NaN,6.5,2.16,6.5,1.85,...,3,under,NaN,NaN,0.15,6.50,6.373709,Under,78.737088,True
13,Luis Severino,severlu,ATH,NaN,NaN,NaN,5.0,1.73,5.5,1.77,...,2,under,NaN,NaN,-0.59,5.25,5.379003,Over,78.709970,True
24,Nathan Eovaldi,eovalna,TEX,NaN,NaN,NaN,5.5,1.83,5.5,2.00,...,2,under,NaN,NaN,0.10,5.50,5.636026,Over,78.639739,True
17,Blake Snell,snellbl,LAD,NaN,NaN,NaN,6.5,1.66,7.0,1.73,...,1,under,NaN,NaN,0.42,6.75,7.001144,Over,77.488561,True
16,Jose Berrios,berrijo,TOR,NaN,NaN,NaN,4.5,1.80,5.0,1.73,...,2,under,NaN,NaN,0.43,4.75,5.070646,Over,76.793537,True
20,Logan Webb,webblo,SFG,NaN,NaN,NaN,4.5,1.64,5.5,1.65,...,2,under,NaN,NaN,-0.29,5.00,5.405555,Over,75.944447,True
3,Michael King,kingmi,SDP,NaN,NaN,NaN,5.0,1.73,5.5,1.85,...,2,under,NaN,NaN,1.13,5.25,5.680701,Over,75.692987,True



Model Comparison Results:


,CV_R2,Test_R2,Test_MAE
GradientBoosting,0.681548,0.851829,0.256330
RandomForest,0.516716,0.805991,0.238238
LightGBM,-1.189565,-0.291659,0.766608
XGBoost,-3.217732,0.771562,0.318093
